# Lab 9
 Solutions

$\color{red}{\text{RULES:}}$

1) Work individually! 

2) You can use the lecture notes (that are posted on D2L), and other materials (e.g. books, web searches).



### Question 1

Consider the task of predicting automobile fuel efficiency, which is characterized by miles per gallon (mpg) metric. The data file "auto_mpg.csv" stores the instances for this task.
- (1.a) Read the csv file, and store it as a pandas dataframe named "df_mpg"
- (1.b) Clean the data, convert categorical features to numerical ones and handle the missing values by replacing them with the column/feature averages (i.e. mode). 
> Hint1: In loan prediction task, for the 'LoanAmount' feature, we used **df_lp['LoanAmount'].fillna(df_lp['LoanAmount'].mode()[0], inplace=True)** to replace missing values. 

> Hint2: In loan prediction task, for the 'Gender' feature, we used **df_lp['Gender'] =df_lp['Gender'].astype('category').cat.codes** to convert categorical feature to numerical one.

- (1.c) separate the class label, mpg, and split the data into 80/20 train test split
- (1.d) train linear regression, knn regression, decision tree regression and random forest regression models. Report the performances of each model (e.g. mean squared error (mse), rmse, and r2_score) over the test set.

### Libraries

In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import datasets, linear_model # linear model includes {LinearRegression, SGDRegressor, Lasso, Ridge, ElasticNet}
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.tree import DecisionTreeClassifier

givenDec = lambda gdVal: float('%.1f' % gdVal) # 1 digit

### Solution 1.a

In [2]:
# read csv
df_mpg = pd.read_csv("auto_mpg.csv")
df_mpg.head()

,cylinders,displacement,horsepower,weight,acceleration,modelYear,origin,carName,mpg
0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,18.0
1,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,15.0
2,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,18.0
3,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,16.0
4,8,302.0,140.0,3449,10.5,70,1,ford torino,17.0


### Solution 1.b

In [3]:
# convert categorical to numerical
df_mpg['carName'] =df_mpg['carName'].astype('category').cat.codes
df_mpg.head()

,cylinders,displacement,horsepower,weight,acceleration,modelYear,origin,carName,mpg
0,8,307.0,130.0,3504,12.0,70,1,49,18.0
1,8,350.0,165.0,3693,11.5,70,1,36,15.0
2,8,318.0,150.0,3436,11.0,70,1,231,18.0
3,8,304.0,150.0,3433,12.0,70,1,14,16.0
4,8,302.0,140.0,3449,10.5,70,1,161,17.0


In [4]:
# missing values
print("\n\n before")
print(df_mpg[df_mpg.isnull().any(axis=1)])

df_mpg['horsepower'].fillna(df_mpg['horsepower'].mode()[0], inplace=True)
print("\n\n after")
print(df_mpg[df_mpg.isnull().any(axis=1)])



 before
     cylinders  displacement  horsepower  weight  acceleration  modelYear  \
32           4          98.0         NaN    2046          19.0         71   
126          6         200.0         NaN    2875          17.0         74   
330          4          85.0         NaN    1835          17.3         80   
336          4         140.0         NaN    2905          14.3         80   
354          4         100.0         NaN    2320          15.8         81   
374          4         151.0         NaN    3035          20.5         82   

     origin  carName   mpg  
32        1      156  25.0  
126       1      150  21.0  
330       2      257  40.9  
336       1      152  23.6  
354       2      255  34.5  
374       1        5  23.0  


 after
Empty DataFrame
Columns: [cylinders, displacement, horsepower, weight, acceleration, modelYear, origin, carName, mpg]
Index: []


### Solution 1.c

In [5]:
# separate the class label
labels_mpg = np.array(df_mpg['mpg'])
df_mpg = df_mpg.drop('mpg', axis=1) # remove rowIDs from the rows
df_mpg.head()

,cylinders,displacement,horsepower,weight,acceleration,modelYear,origin,carName
0,8,307.0,130.0,3504,12.0,70,1,49
1,8,350.0,165.0,3693,11.5,70,1,36
2,8,318.0,150.0,3436,11.0,70,1,231
3,8,304.0,150.0,3433,12.0,70,1,14
4,8,302.0,140.0,3449,10.5,70,1,161


In [6]:
# get train/test
# setting the random state to 42 which means the results will be the same each time I run the split for reproducible results.
trSet_mpg, testSet_mpg, trainLab_mpg, testLab_mpg = train_test_split(df_mpg, labels_mpg, test_size = 0.20, random_state = 42)

print('Training instances Shape:', trSet_mpg.shape)
print('Training Labels Shape:', trainLab_mpg.shape)
print('Testing instances Shape:', testSet_mpg.shape)
print('Testing Labels Shape:', testLab_mpg.shape)

Training instances Shape: (318, 8)
Training Labels Shape: (318,)
Testing instances Shape: (80, 8)
Testing Labels Shape: (80,)


### Solution 1.d

In [7]:
def evaluate_results(gTestLabel, gTestPredictions):    
    mse =  round(mean_squared_error(gTestLabel, gTestPredictions), 2)
    rmse = givenDec(math.sqrt(mse))
    r2Score_val = round(r2_score(gTestLabel, gTestPredictions), 2)

    print("mse " + str(mse))
    print("rmse " + str(rmse))
    print("r2Score_val " + str(r2Score_val))
    
    #print('\n')
    #for i in range(len(gTestLabel)):
    #    print("row " + str(i) + '\t' + str(gTestLabel[i]) + '\t' + str(givenDec(gTestPredictions[i])))

#### Linear regression model

In [8]:
regr = linear_model.LinearRegression()
model = regr.fit(trSet_mpg, trainLab_mpg)

# Use the predict method on the test data
test_predictions = model.predict(testSet_mpg)

evaluate_results(testLab_mpg, test_predictions)

mse 8.26
rmse 2.9
r2Score_val 0.85


#### KNN

In [9]:
regr = KNeighborsRegressor(n_neighbors=4) # n_neighbors selection is arbitrary
model = regr.fit(trSet_mpg, trainLab_mpg)

# Use the predict method on the test data
test_predictions = model.predict(testSet_mpg)

evaluate_results(testLab_mpg, test_predictions)

mse 13.09
rmse 3.6
r2Score_val 0.76


#### DT

In [10]:
regr = DecisionTreeRegressor(random_state=42)
model = regr.fit(trSet_mpg, trainLab_mpg)

# Use the predict method on the test data
test_predictions = model.predict(testSet_mpg)

evaluate_results(testLab_mpg, test_predictions)

mse 9.76
rmse 3.1
r2Score_val 0.82


#### RF

In [11]:
regr = RandomForestRegressor(random_state=42, n_estimators=10)
model = regr.fit(trSet_mpg, trainLab_mpg)

# Use the predict method on the test data
test_predictions = model.predict(testSet_mpg)

evaluate_results(testLab_mpg, test_predictions)

mse 4.98
rmse 2.2
r2Score_val 0.91
